In [2]:
import pandas as pd
import os

# --- 1. 配置参数 ---
# 请根据您的实际情况修改文件路径和工作表名称
file_path = r"E:\A智网\8.27湖北重点行业变化分析\分省重点行业数据.xlsx"
source_sheet_name = "data"
output_sheet_name = "重点行业分析结果"

# --- 2. 读取和预处理数据 ---
try:
    print(f"正在读取文件: {file_path}")
    # 读取指定sheet的数据
    df = pd.read_excel(file_path, sheet_name=source_sheet_name)
    print("文件读取成功，开始处理数据...")

    # 自动识别月份字段（例如 '202503', '202504' 等）
    # 假设月份字段是所有6位数的数字列
    month_cols = [col for col in df.columns if isinstance(col, (int, str)) and str(col).isdigit() and len(str(col)) == 6]
    
    # 将月份字段统一转换为字符串，防止后续操作因类型问题出错
    df.rename(columns={col: str(col) for col in month_cols}, inplace=True)
    month_cols = [str(col) for col in month_cols]

    # 清理可能存在的空格
    df['省份'] = df['省份'].str.strip()
    df['行业名称'] = df['行业名称'].str.strip()

    # 计算用于排序的“3-8月电量总和”
    df['总电量'] = df[month_cols].sum(axis=1)

    # --- 3. 计算“全国”数据 ---
    # 按行业分组，对每个月的电量求和
    national_df = df.groupby(['行业序号', '行业名称'])[month_cols].sum().reset_index()
    # 添加“省份”列，并赋值为“全国”
    national_df['省份'] = '全国'
    print("已计算各行业全国总电量。")

    # --- 4. 筛选各行业Top5省份及湖北省数据 ---
    # 创建一个空的DataFrame用于存放最终筛选结果
    top_provinces_list = []

    # 按“行业序号”和“行业名称”进行分组
    for (industry_code, industry_name), group in df.groupby(['行业序号', '行业名称']):
        # 对每个行业内部的省份按“总电量”降序排列
        group_sorted = group.sort_values(by='总电量', ascending=False)
        
        # 提取电量前五的省份
        top5 = group_sorted.head(5)
        
        # 提取湖北省的数据
        hubei_data = group[group['省份'] == '湖北']
        
        # 检查湖北省是否已在前五名中
        is_hubei_in_top5 = '湖北' in top5['省份'].values
        
        # 如果湖北省不在前五且存在湖北省数据，则将其添加进去
        if not is_hubei_in_top5 and not hubei_data.empty:
            final_group = pd.concat([top5, hubei_data])
        else:
            final_group = top5
            
        top_provinces_list.append(final_group)

    # 将所有行业的筛选结果合并成一个DataFrame
    top_provinces_df = pd.concat(top_provinces_list)
    print("已筛选出各行业Top5省份及湖北省数据。")

    # --- 5. 合并与整理最终结果 ---
    # 定义最终输出的列顺序
    output_columns = ['省份', '行业序号', '行业名称'] + month_cols
    
    # 组合“全国”数据和“各省”数据，并只保留需要的列
    final_df = pd.concat([national_df, top_provinces_df])[output_columns]

    # 为了美观，对结果进行排序：按行业序号升序，在每个行业内，“全国”排第一，其余按总电量降序
    # 重新计算总电量用于排序
    final_df['总电量'] = final_df[month_cols].sum(axis=1)
    # 创建一个辅助列用于排序，“全国”为0，其他为1
    final_df['排序辅助'] = final_df['省份'].apply(lambda x: 0 if x == '全国' else 1)
    
    # 执行排序
    final_df_sorted = final_df.sort_values(
        by=['行业序号', '排序辅助', '总电量'],
        ascending=[True, True, False]
    )

    # 删除用于排序的辅助列
    final_df_sorted = final_df_sorted.drop(columns=['总电量', '排序辅助'])
    print("数据整合与排序完成。")

    # --- 6. 写入到新的Excel Sheet ---
    print(f"正在将结果写入到文件: {file_path} 的新工作表: {output_sheet_name}")
    # 使用 ExcelWriter 以“追加”模式打开文件，如果目标sheet已存在则替换它
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        final_df_sorted.to_excel(writer, sheet_name=output_sheet_name, index=False)

    print("--- 操作成功！---")
    print(f"结果已保存在 '{os.path.basename(file_path)}' 文件的 '{output_sheet_name}' 工作表中。")

except FileNotFoundError:
    print(f"错误：文件未找到，请检查路径是否正确: {file_path}")
except Exception as e:
    print(f"处理过程中发生错误: {e}")

正在读取文件: E:\A智网\8.27湖北重点行业变化分析\分省重点行业数据.xlsx
文件读取成功，开始处理数据...
已计算各行业全国总电量。
已筛选出各行业Top5省份及湖北省数据。
数据整合与排序完成。
正在将结果写入到文件: E:\A智网\8.27湖北重点行业变化分析\分省重点行业数据.xlsx 的新工作表: 重点行业分析结果
--- 操作成功！---
结果已保存在 '分省重点行业数据.xlsx' 文件的 '重点行业分析结果' 工作表中。


In [20]:
import pandas as pd
import numpy as np
import os

# --- 1. 配置参数 ---
# 文件路径保持不变
file_path = r"E:\A智网\8.27湖北重点行业变化分析\data.xlsx"
# 源工作表现在是上一步的结果
source_sheet_name = "25天版"
# 输出一个新的工作表，名称清晰
output_sheet_name = "累计环比增速_8月25天"

# --- 2. 读取预处理好的数据 ---
try:
    print(f"正在读取已处理好的数据: '{source_sheet_name}' sheet...")
    df = pd.read_excel(file_path, sheet_name=source_sheet_name)
    print("数据读取成功，开始计算累计环比增速...")

    # --- 3. 识别列并确定基期 ---
    # 自动识别月份字段
    month_cols = [col for col in df.columns if isinstance(col, (int, str)) and str(col).isdigit() and len(str(col)) == 6]
    
    # 将月份字段统一为字符串并排序，确保时间顺序
    df.rename(columns={col: str(col) for col in month_cols}, inplace=True)
    month_cols = sorted([str(col) for col in month_cols]) # 排序以确定第一个月为基期

    # 确定基期月份（第一个月）
    base_month = month_cols[0]
    print(f"已确定基期月份为: {base_month}")

    # 识别标识列
    id_cols = ['省份', '行业序号', '行业名称']

    # --- 4. 核心计算：累计环比增速 ---
    # 创建一个用于存放结果的DataFrame，先复制标识列
    result_df = df[id_cols].copy()

    # 基期（第一个月）的增速定义为0，这里用'-'表示
    result_df[base_month] = '-'

    # 遍历从第二个月开始的所有月份进行计算
    for month in month_cols[1:]:
        # 获取基期和当期的用电量数据
        base_values = df[base_month]
        current_values = df[month]
        
        # 计算公式：(当月值 - 基期值) / 基期值
        # 使用 np.where 处理基期值为0的特殊情况，避免除以零错误
        growth_rate = np.where(
            base_values == 0, 
            np.nan,  # 如果基期值为0，则增长率无意义，设为空值(NaN)
            (current_values - base_values) / base_values
        )
        result_df[month] = growth_rate
    
    print("累计环比增速计算完成，正在格式化输出...")

    # --- 5. 格式化输出 ---
    # 遍历所有计算出的月份列，将小数格式化为百分比字符串
    for col in month_cols[1:]:
        result_df[col] = result_df[col].apply(
            lambda x: f"{x:.2%}" if pd.notna(x) else '-'
        )
    
    # 因为源数据已经排好序，所以这里不需要再排序了
    final_df = result_df

    # --- 6. 写入到新的Excel Sheet ---
    print(f"正在将结果写入到文件: {file_path} 的新工作表: {output_sheet_name}")
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        final_df.to_excel(writer, sheet_name=output_sheet_name, index=False)

    print("--- 操作成功！---")
    print(f"基于分析结果计算的累计环比增速已保存在 '{os.path.basename(file_path)}' 文件的 '{output_sheet_name}' 工作表中。")

except FileNotFoundError:
    print(f"错误：文件未找到，请检查路径是否正确: {file_path}")
except ValueError as e:
    if f"Worksheet '{source_sheet_name}' not found" in str(e):
        print(f"错误：找不到名为 '{source_sheet_name}' 的工作表。请先运行第一个脚本生成该表。")
    else:
        print(f"处理过程中发生值错误: {e}")
except Exception as e:
    print(f"处理过程中发生未知错误: {e}")

正在读取已处理好的数据: '25天版' sheet...
数据读取成功，开始计算累计环比增速...
已确定基期月份为: 202503
累计环比增速计算完成，正在格式化输出...
正在将结果写入到文件: E:\A智网\8.27湖北重点行业变化分析\data.xlsx 的新工作表: 累计环比增速_8月25天
--- 操作成功！---
基于分析结果计算的累计环比增速已保存在 'data.xlsx' 文件的 '累计环比增速_8月25天' 工作表中。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import PercentFormatter

# --- 1. 配置参数 ---
file_path = r"E:\A智网\8.27湖北重点行业变化分析\日均累计环比增速.xlsx"
# 读取上一步生成的、包含累计环比增速的工作表
source_sheet_name = "26天"
# 创建一个文件夹来存放所有生成的图表
output_chart_folder = r"E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_26天"

# --- 2. 数据加载与预处理 ---
try:
    print(f"正在读取数据: '{source_sheet_name}' sheet...")
    df = pd.read_excel(file_path, sheet_name=source_sheet_name)
    print("数据读取成功。")

    # 创建输出图表的文件夹（如果不存在）
    if not os.path.exists(output_chart_folder):
        os.makedirs(output_chart_folder)
        print(f"已创建文件夹: '{output_chart_folder}' 用于存放图表。")

    # 识别月份列和标识列
    month_cols = [col for col in df.columns if isinstance(col, (int, str)) and str(col).isdigit() and len(str(col)) == 6]
    month_cols = sorted([str(col) for col in month_cols])

    # **关键步骤：将百分比字符串转换回可供绘图的数值**
    def convert_to_float(value):
        """将 '5.21%' 或 '-' 这样的字符串转换为浮点数 0.0521 或 0.0"""
        if isinstance(value, str):
            if value.strip() == '-':
                return 0.0
            try:
                # 移除 '%' 符号，转换为浮点数，然后除以100
                return float(value.strip().replace('%', '')) / 100
            except ValueError:
                return 0.0
        # 如果已经是数字，直接返回
        return float(value)

    # 对所有月份列应用这个转换函数
    for col in month_cols:
        df[col] = df[col].apply(convert_to_float)
    
    print("数据已成功转换为数值格式，准备绘图...")

    # 设置matplotlib的中文字体，以防图表中的中文显示为方框
    # 你可以根据你的操作系统选择一个已安装的中文字体
    plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows系统常用'SimHei'（黑体）
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号'-'显示为方块的问题

    # --- 3. 循环为每个行业绘制并保存图表 ---
    # 获取所有唯一的行业名称
    unique_industries = df['行业名称'].unique()

    for industry in unique_industries:
        print(f"正在为行业 '{industry}' 生成图表...")
        
        # 筛选出当前行业的数据
        industry_df = df[df['行业名称'] == industry]

        # 创建一个新的图表和坐标轴
        fig, ax = plt.subplots(figsize=(12, 7))

        # 遍历该行业内的每个省份（或全国）并绘制折线
        for index, row in industry_df.iterrows():
            province = row['省份']
            growth_data = row[month_cols]
            
            # --- 核心：对湖北省进行特殊处理 ---
            if province == '湖北':
                ax.plot(month_cols, growth_data, 
                        label=province, 
                        linewidth=3.5,  # 标粗线条
                        marker='o',       # 添加标记点
                        markersize=6,
                        color='red',      # 使用醒目的红色
                        zorder=10)        # zorder确保该线条在最上层
            elif province == '全国':
                ax.plot(month_cols, growth_data, 
                        label=province, 
                        linewidth=2.0, 
                        linestyle='--', # 使用虚线区分“全国”
                        marker='s',
                        zorder=5)
            else:
                ax.plot(month_cols, growth_data, 
                        label=province, 
                        linewidth=1.5,
                        marker='.',
                        alpha=0.8) # 其他省份线条稍细，略带透明

        # --- 4. 美化图表 ---
        # 设置图表标题
        ax.set_title(f'{industry}业累计环比增速趋势对比', fontsize=16, pad=20)
        
        # 设置坐标轴标签
        ax.set_xlabel('月份', fontsize=12)
        ax.set_ylabel('日均累计环比增速 (基期: ' + month_cols[0] + ')', fontsize=12)
        
        # 将Y轴格式化为百分比显示
        ax.yaxis.set_major_formatter(PercentFormatter(1.0))
        
        # 添加图例
        ax.legend(title='省份/区域', bbox_to_anchor=(1.02, 1), loc='upper left')
        
        # 添加网格线
        ax.grid(True, linestyle='--', alpha=0.6)
        
        # 旋转X轴标签，防止重叠
        plt.xticks(rotation=45)
        
        # 调整布局以防止标签被截断
        plt.tight_layout(rect=[0, 0, 0.9, 1]) # 为图例留出右侧空间

        # --- 5. 保存图表 ---
        # 创建一个对文件名安全的文件名
        safe_industry_name = industry.replace('/', '_').replace('\\', '_')
        chart_filename = os.path.join(output_chart_folder, f'{safe_industry_name}_增速趋势图.png')
        
        plt.savefig(chart_filename, dpi=600) # dpi=300 保存为高分辨率图片
        
        # 关闭当前图表，释放内存，准备绘制下一张
        plt.close(fig)
        
        print(f"图表已保存至: {chart_filename}")

    print("\n--- 所有图表生成完毕！---")

except FileNotFoundError:
    print(f"错误：文件未找到，请检查路径是否正确: {file_path}")
except ValueError as e:
    if f"Worksheet '{source_sheet_name}' not found" in str(e):
        print(f"错误：找不到名为 '{source_sheet_name}' 的工作表。请先运行前序脚本生成该表。")
    else:
        print(f"处理过程中发生值错误: {e}")
except Exception as e:
    print(f"处理过程中发生未知错误: {e}")

正在读取数据: '25天' sheet...
数据读取成功。
已创建文件夹: 'E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天' 用于存放图表。
数据已成功转换为数值格式，准备绘图...
正在为行业 '纺织' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\纺织_增速趋势图.png
正在为行业 '化工' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\化工_增速趋势图.png
正在为行业 '医药制造' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\医药制造_增速趋势图.png
正在为行业 '建材' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\建材_增速趋势图.png
正在为行业 '钢铁' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\钢铁_增速趋势图.png
正在为行业 '有色金属' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\有色金属_增速趋势图.png
正在为行业 '金属制品' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\金属制品_增速趋势图.png
正在为行业 '通用设备' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\通用设备_增速趋势图.png
正在为行业 '专用设备' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\专用设备_增速趋势图.png
正在为行业 '汽车' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\汽车_增速趋势图.png
正在为行业 '铁路船舶' 生成图表...
图表已保存至: E:\A智网\8.27湖北重点行业变化分析\重点行业增速趋势图_25天\铁路船舶_增速趋势图.png
正在为行业 '电气机械' 生成图表...
图表已保存至: E:\A智